<a href="https://colab.research.google.com/github/seahahn/sec4/blob/main/chatbot_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoGPT2 모델을 이용하여 보다 자연스러운 챗봇 만들어보기

## 1. 데이터셋 가져오기

In [1]:
import pandas as pd
import re
import urllib.request

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1-1. Chatbot_data
출처 : https://github.com/songys/Chatbot_data

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
cbdata_df = pd.read_csv('ChatBotData.csv')
cbdata_df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


### 1-2. 트위터에서 수집 및 정제한 대화 시나리오

출처 : [트위터에서 수집 및 정제한 대화 시나리오 - AI Hub](https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-008)

In [ ]:
tt_df = pd.read_excel('/content/drive/MyDrive/cd_ai/s4p/twitter_talk.xlsx')
tt_df.head()

In [ ]:
tt_df = tt_df.dropna(axis='columns', how="all")
tt_df.head()

In [ ]:
len(tt_df.columns)

83

In [ ]:
[tt_df.iloc[0, 0], tt_df.iloc[0, 1]]

['기계식 키보드 써보고 싶어요.', '싼 것도 많던데 써보세요. 정말 좋아요.']

In [ ]:
columns = ['Q', 'A', 'label']
twitter_talk = pd.DataFrame(columns=columns)

for row in range(len(tt_df.index)):
  if row % 500 == 0: print(f'row : {row}')
  for col in range(len(tt_df.columns)):
    if col+1 == len(tt_df.columns):
      break
    if pd.isna(tt_df.iloc[row, col+1]):
      break
    sent_couple = pd.DataFrame([[tt_df.iloc[row, col], tt_df.iloc[row, col+1], 0]], columns=columns)
    twitter_talk = twitter_talk.append(sent_couple, ignore_index=True)

row : 0
row : 500
row : 1000
row : 1500


In [ ]:
twitter_talk.head()

,Q,A,label
0,기계식 키보드 써보고 싶어요.,싼 것도 많던데 써보세요. 정말 좋아요.,0
1,싼 것도 많던데 써보세요. 정말 좋아요.,싼 것도 있나요? 예전에 봤을 땐 다 10만 원이 넘던데요.,0
2,싼 것도 있나요? 예전에 봤을 땐 다 10만 원이 넘던데요.,이거 쓰는데 예쁘고 딸깍거려요.,0
3,이거 쓰는데 예쁘고 딸깍거려요.,우와 진짜 예뻐요.,0
4,우와 진짜 예뻐요.,불 끄면 더 예뻐요.,0


### 1-3. 한국어 감정 정보가 포함된 연속적 대화 데이터셋

출처 : [한국어 감정 정보가 포함된 연속적 대화 데이터셋 - AI Hub](https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-010)

In [ ]:
ket_df = pd.read_excel('/content/drive/MyDrive/cd_ai/s4p/kor_emo_talk.xlsx')
ket_df.head()

,dialog #,발화,감정
0,S,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
1,NaN,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
2,NaN,손님 왔어요.,중립
3,NaN,손님? 누구?,중립
4,NaN,몰라요. 팀장님 친구래요.,중립


In [ ]:
[ket_df.iloc[0, 1], ket_df.iloc[1, 1]]

['아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!',
 '그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.']

In [ ]:
ket_df['감정'].unique()

array(['분노', '혐오', '중립', '놀람', '행복', '공포', '슬픔', 'ㅈ중립', '분ㄴ', '중림', nan,
       'ㅍ', 'ㄴ중립', '분', '줄'], dtype=object)

In [ ]:
ket_df[(ket_df['감정'] == 'ㅈ중립') | (ket_df['감정'] == '중림') | (ket_df['감정'] == '중림') | (ket_df['감정'] == 'ㄴ중립') | (ket_df['감정'] == '줄')] = '중립'
ket_df[(ket_df['감정'] == '분ㄴ') | (ket_df['감정'] == '분')] = '분노'
ket_df[ket_df['감정'] == 'ㅍ'] = '공포'

In [ ]:
ket_df[ket_df['감정'].isna()] = '중립' # NaN을 없애버리면 맥락을 잃을 수 있음. 대신 중립으로 설정

In [ ]:
ket_df['감정'].unique()

array(['분노', '혐오', '중립', '놀람', '행복', '공포', '슬픔'], dtype=object)

In [ ]:
ket_df[ket_df['감정'] == '놀람']

,dialog #,발화,감정
16,NaN,나? ... 나보고 하라고?,놀람
18,NaN,근데 왜... 나한테...?,놀람
48,NaN,... 오.,놀람
59,NaN,... 대리석?,놀람
105,NaN,요기잖아. 요기. 강 건너 윗동네도 몰라?,놀람
...,...,...,...
55464,NaN,...그거 괜찮긴 한 거야?,놀람
55523,NaN,……이 인형은 뭐야?,놀람
55585,NaN,어? 정말요?,놀람
55587,NaN,"혹시, 다들 은행 계좌없는 거예요?",놀람


In [ ]:
# 중립 (0), 부정 (1), 긍정 (2)
neg_emotion = ['분노', '혐오', '공포', '슬픔']
pos_emotion = ['행복']
neutral_emotion = ['놀람', '중립'] # 놀람은 긍정 또는 부정 한쪽으로 구분하기 어려운 관계로 중립에 포함

ket_df['감정'][ket_df['감정'].isin(neg_emotion)] = 1
ket_df['감정'][ket_df['감정'].isin(pos_emotion)] = 2
ket_df['감정'][ket_df['감정'].isin(neutral_emotion)] = 0

In [ ]:
ket_df.iloc[0,0] = 'start' # 시작 지점 설정(이후 동일한 열에서 새로운 맥락이 시작되는 지점은 S로 표시되어 있음)

In [ ]:
ket_df.head()

,dialog #,발화,감정
0,start,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,1
1,NaN,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,1
2,NaN,손님 왔어요.,0
3,NaN,손님? 누구?,0
4,NaN,몰라요. 팀장님 친구래요.,0


In [ ]:
columns = ['Q', 'A', 'label']
kor_emo_talk = pd.DataFrame(columns=columns)

for row in range(len(ket_df.index)):
  try:
    if row % 5000 == 0: print(f'row : {row}')
    if ket_df.iloc[row+1, 0] == 'S' : continue # NaN이 아닌 값(새로운 맥락의 대화)이 나오면 다음 순으로 넘기기
    sent_couple = pd.DataFrame([[ket_df.iloc[row, 1], ket_df.iloc[row+1, 1], ket_df.iloc[row+1, 2]]], columns=columns)
    kor_emo_talk = kor_emo_talk.append(sent_couple, ignore_index=True)
  except:
    print('done')

row : 0
row : 5000
row : 10000
row : 15000
row : 20000
row : 25000
row : 30000
row : 35000
row : 40000
row : 45000
row : 50000
row : 55000
done


In [ ]:
kor_emo_talk.head()

,Q,A,label
0,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,1
1,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,손님 왔어요.,0
2,손님 왔어요.,손님? 누구?,0
3,손님? 누구?,몰라요. 팀장님 친구래요.,0
4,몰라요. 팀장님 친구래요.,내 친구? 친구 누구?,0


In [ ]:
kor_emo_talk[['Q', 'A']] = kor_emo_talk[['Q', 'A']].applymap(lambda x: x.replace(u'\xa0', u' '))

### 1-4. 한국어 대화 데이터셋

출처 : [한국어 대화 데이터셋 - AI Hub](https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-011)

In [ ]:
# 응급 상황 대화
te_df = pd.read_excel('/content/drive/MyDrive/cd_ai/s4p/talk_emergency.xlsx')
te_df.head(25)

,num,talk
0,1,﻿아빠가 숨을 잘 못쉬어요.
1,2,그래요. 주소가 어떻게 되나요?
2,3,한남동 123번지 고마스빌딩이예요.
3,4,가슴에 통증이 있는지 물어봐줄래요?
4,5,있다고 해요. 이런경우는 처음이래요.
5,6,잠시만 기다려주세요. 도와줄 사람들을 금방 보내줄게요.알겠죠?
6,7,알겠어요. 빨리 오셔야 되요.
7,8,지금 사람들이 가고 있어요. 아버지는 지금 깨어있나요?
8,9,네
9,10,전에도 이런적이 있었는지 아빠한테 물어봐줄래요?


In [ ]:
te_df.iloc[0,0] = 'start'
te_df.head()

,num,talk
0,start,﻿아빠가 숨을 잘 못쉬어요.
1,2,그래요. 주소가 어떻게 되나요?
2,3,한남동 123번지 고마스빌딩이예요.
3,4,가슴에 통증이 있는지 물어봐줄래요?
4,5,있다고 해요. 이런경우는 처음이래요.


In [ ]:
columns = ['Q', 'A', 'label']
talk_emergency = pd.DataFrame(columns=columns)

for row in range(len(te_df.index)):
  try:
    if row % 1000 == 0: print(f'row : {row}')
    if te_df.iloc[row+1, 0] == 1 : continue
    sent_couple = pd.DataFrame([[te_df.iloc[row, 1], te_df.iloc[row+1, 1], 0]], columns=columns)
    talk_emergency = talk_emergency.append(sent_couple, ignore_index=True)
  except:
    print('done')

row : 0
row : 1000
row : 2000
row : 3000
row : 4000
done


In [ ]:
talk_emergency.head()

,Q,A,label
0,﻿아빠가 숨을 잘 못쉬어요.,그래요. 주소가 어떻게 되나요?,0
1,그래요. 주소가 어떻게 되나요?,한남동 123번지 고마스빌딩이예요.,0
2,한남동 123번지 고마스빌딩이예요.,가슴에 통증이 있는지 물어봐줄래요?,0
3,가슴에 통증이 있는지 물어봐줄래요?,있다고 해요. 이런경우는 처음이래요.,0
4,있다고 해요. 이런경우는 처음이래요.,잠시만 기다려주세요. 도와줄 사람들을 금방 보내줄게요.알겠죠?,0


In [ ]:
# 일상(회사) 대화
to_df = pd.read_excel('/content/drive/MyDrive/cd_ai/s4p/talk_office.xlsx')
to_df.head()

,num,talk
0,1,﻿좋은 아침.
1,2,안녕하세요.
2,1,좋은 아침.
3,2,반갑습니다.
4,1,좋은 아침.


In [ ]:
columns = ['Q', 'A', 'label']
talk_office = pd.DataFrame(columns=columns)

for row in range(len(to_df.index)):
  try:
    if row % 1000 == 0: print(f'row : {row}')
    if row % 2 == 0:
      sent_couple = pd.DataFrame([[to_df.iloc[row, 1], to_df.iloc[row+1, 1], 0]], columns=columns)
      talk_office = talk_office.append(sent_couple, ignore_index=True)
  except:
    print('done')

row : 0
row : 1000
row : 2000


In [ ]:
talk_office.head()

,Q,A,label
0,﻿좋은 아침.,안녕하세요.,0
1,좋은 아침.,반갑습니다.,0
2,좋은 아침.,좋은 아침이에요.,0
3,좋은 아침.,간밤에 별일 없으셨죠?,0
4,좋은 아침.,안녕하시렵니까?,0


### 1-5. 데이터셋 합치기

In [ ]:
df = pd.concat([cbdata_df, twitter_talk, kor_emo_talk, talk_emergency, talk_office], ignore_index=True)

In [ ]:
df.shape

(74588, 3)

In [ ]:
df.to_csv('data_total.csv', encoding='utf-8')

## 2, 모델 학습시키기

In [3]:
# GPU 정보 
!nvidia-smi

Wed Jan 12 09:07:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# KoGPT2-chatbot 소스 코드 복사
!git clone --recurse-submodules https://github.com/seahahn/KoGPT2-chatbot.git

Cloning into 'KoGPT2-chatbot'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 106 (delta 9), reused 8 (delta 3), pack-reused 85
Receiving objects: 100% (106/106), 161.32 KiB | 11.52 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Submodule 'Chatbot_data' (https://github.com/haven-jeon/Chatbot_data.git) registered for path 'Chatbot_data'
Cloning into '/content/KoGPT2-chatbot/Chatbot_data'...
remote: Enumerating objects: 24, done.        
remote: Counting objects: 100% (4/4), done.        
remote: Compressing objects: 100% (4/4), done.        
remote: Total 24 (delta 0), reused 3 (delta 0), pack-reused 20        
Submodule path 'Chatbot_data': checked out '235fac5aea3badab22743f7048afe936cf72f822'


In [5]:
# 폴더 이동
%cd KoGPT2-chatbot

/content/KoGPT2-chatbot


In [ ]:
!pip install -r requirements.txt

In [ ]:
# 사전훈련된 KoGPT2를 챗봇 데이터로 파인튜닝
!CUDA_VISIBLE_DEVICES=0 python train_torch.py --train --gpus 1 --max_epochs 100 --batch-size 128 --data_path '/content/data_total.csv'

In [ ]:
# 대화 테스트, `quit`를 입력하면 대화 종료
# 미리 학습해둔 가중치를 불러옴
# 만약 새로 학습한 가중치를 불러오려면 ckpt 파일의 경로를 해당 파일 경로로 지정해주면 됨
!CUDA_VISIBLE_DEVICES=0 python train_torch.py --gpus 1 --model_params '/content/drive/MyDrive/cd_ai/s4p/model_best.ckpt' --chat

INFO:root:Namespace(accelerator=None, accumulate_grad_batches=1, amp_backend='native', amp_level='O2', auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, automatic_optimization=None, batch_size=96, benchmark=False, chat=True, check_val_every_n_epoch=1, checkpoint_callback=True, data_path='Chatbot_data/ChatbotData.csv', default_root_dir=None, deterministic=False, distributed_backend=None, enable_pl_optimizer=None, fast_dev_run=False, flush_logs_every_n_steps=100, gpus=1, gradient_clip_val=0, limit_predict_batches=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=5e-05, max_epochs=None, max_len=32, max_steps=None, min_epochs=None, min_steps=None, model_params='/content/drive/MyDrive/cd_ai/s4p/model_best.ckpt', move_metrics_to_cpu=False, multiple_trainloader_mode='max_size_cycle', num_nodes=1, num_processes=1, num_sanity_val_steps=2, overfit_batches=0.0, plugins=None, precision=32, 

## 3. 모델 학습 가중치를 pt 파일로 만들기

### 3-1. 기존 train_torch.py 파일의 내용 가져와서 그대로 초기화

In [21]:
import argparse
import logging

import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

parser = argparse.ArgumentParser(description='Simsimi based on KoGPT-2')

parser.add_argument('--chat',
                    action='store_true',
                    default=False,
                    help='response generation on given user input')

parser.add_argument('--sentiment',
                    type=str,
                    default='0',
                    help='sentiment for system. 0 is neutral, 1 is negative, 2 is positive.')

parser.add_argument('--model_params',
                    type=str,
                    default='model_chp/model_-last.ckpt',
                    help='model binary for starting chat')

parser.add_argument('--train',
                    action='store_true',
                    default=False,
                    help='for training')

parser.add_argument('--data_path',
                    type=str,
                    default='Chatbot_data/ChatbotData.csv',
                    help='training data path')

logger = logging.getLogger()
logger.setLevel(logging.INFO)

U_TKN = '<usr>'
S_TKN = '<sys>'
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 

In [22]:
class CharDataset(Dataset):
    def __init__(self, chats, max_len=32):
        self._data = chats
        self.first = True
        self.q_token = U_TKN
        self.a_token = S_TKN
        self.sent_token = SENT
        self.bos = BOS
        self.eos = EOS
        self.mask = MASK
        self.pad = PAD
        self.max_len = max_len
        self.tokenizer = TOKENIZER 

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        turn = self._data.iloc[idx]
        q = turn['Q']
        a = turn['A']
        sentiment = str(turn['label'])
        q_toked = self.tokenizer.tokenize(self.q_token + q + \
                                          self.sent_token + sentiment)   
        q_len = len(q_toked)
        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len
            if a_len <= 0:
                q_toked = q_toked[-(int(self.max_len/2)):]
                q_len = len(q_toked)
                a_len = self.max_len - q_len
                assert a_len > 0
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)
            assert a_len == len(a_toked), f'{a_len} ==? {len(a_toked)}'
        # [mask, mask, ...., mask, ..., <bos>,..A.. <eos>, <pad>....]
        labels = [
            self.mask,
        ] * q_len + a_toked[1:]
        if self.first:
            logging.info("contexts : {}".format(q))
            logging.info("toked ctx: {}".format(q_toked))
            logging.info("response : {}".format(a))
            logging.info("toked response : {}".format(a_toked))
            logging.info('labels {}'.format(labels))
            self.first = False
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        self.max_len
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]
        return(token_ids, np.array(mask),
               labels_ids)

In [23]:
class KoGPT2Chat(LightningModule):
    def __init__(self, hparams, **kwargs):
        super(KoGPT2Chat, self).__init__()
        self.hparams = hparams
        self.neg = -1e18
        self.kogpt2 = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
        self.loss_function = torch.nn.CrossEntropyLoss(reduction='none')

    @staticmethod
    def add_model_specific_args(parent_parser):
        # add model specific args
        parser = argparse.ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument('--max-len',
                            type=int,
                            default=32,
                            help='max sentence length on input (default: 32)')

        parser.add_argument('--batch-size',
                            type=int,
                            default=96,
                            help='batch size for training (default: 96)')
        parser.add_argument('--lr',
                            type=float,
                            default=5e-5,
                            help='The initial learning rate')
        parser.add_argument('--warmup_ratio',
                            type=float,
                            default=0.1,
                            help='warmup ratio')
        return parser

    def forward(self, inputs):
        # (batch, seq_len, hiddens)
        output = self.kogpt2(inputs, return_dict=True)
        return output.logits

    def training_step(self, batch, batch_idx):
        token_ids, mask, label = batch
        out = self(token_ids)
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, self.neg * torch.ones_like(out))
        loss = self.loss_function(mask_out.transpose(2, 1), label)
        loss_avg = loss.sum() / mask.sum()
        self.log('train_loss', loss_avg)
        return loss_avg

    def configure_optimizers(self):
        # Prepare optimizer
        param_optimizer = list(self.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=self.hparams.lr, correct_bias=False)
        # warm up lr
        num_train_steps = len(self.train_dataloader()) * self.hparams.max_epochs
        num_warmup_steps = int(num_train_steps * self.hparams.warmup_ratio)
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)
        lr_scheduler = {'scheduler': scheduler, 'name': 'cosine_schedule_with_warmup',
                        'monitor': 'loss', 'interval': 'step',
                        'frequency': 1}
        return [optimizer], [lr_scheduler]

    def _collate_fn(self, batch):
        data = [item[0] for item in batch]
        mask = [item[1] for item in batch]
        label = [item[2] for item in batch]
        return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

    def train_dataloader(self):
        data = pd.read_csv(self.hparams.data_path)
        self.train_set = CharDataset(data, max_len=self.hparams.max_len)
        train_dataloader = DataLoader(
            self.train_set, batch_size=self.hparams.batch_size, num_workers=2,
            shuffle=True, collate_fn=self._collate_fn)
        return train_dataloader

    def chat(self, sent='0'):
        tok = TOKENIZER
        sent_tokens = tok.tokenize(sent)
        with torch.no_grad():
            while 1:
                q = input('user > ').strip()
                if q == 'quit':
                    break
                a = ''
                while 1:
                    input_ids = torch.LongTensor(tok.encode(U_TKN + q + SENT + sent + S_TKN + a)).unsqueeze(dim=0)
                    pred = self(input_ids)
                    gen = tok.convert_ids_to_tokens(
                        torch.argmax(
                            pred,
                            dim=-1).squeeze().numpy().tolist())[-1]
                    if gen == EOS:
                        break
                    a += gen.replace('▁', ' ')
                print("Simsimi > {}".format(a.strip()))

colab에서는 parser.parse_args()에서 아래와 같은 에러가 발생.  
대신 easydict를 이용하여 파라미터를 전달 가능

In [66]:
# parser = KoGPT2Chat.add_model_specific_args(parser)
# parser = Trainer.add_argparse_args(parser)
# parser.parse_args()
# args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--chat] [--sentiment SENTIMENT]
                             [--model_params MODEL_PARAMS] [--train]
                             [--data_path DATA_PATH] [--max-len MAX_LEN]
                             [--batch-size BATCH_SIZE] [--lr LR]
                             [--warmup_ratio WARMUP_RATIO] [--logger [LOGGER]]
                             [--checkpoint_callback [CHECKPOINT_CALLBACK]]
                             [--default_root_dir DEFAULT_ROOT_DIR]
                             [--gradient_clip_val GRADIENT_CLIP_VAL]
                             [--process_position PROCESS_POSITION]
                             [--num_nodes NUM_NODES]
                             [--num_processes NUM_PROCESSES] [--gpus GPUS]
                             [--auto_select_gpus [AUTO_SELECT_GPUS]]
                             [--tpu_cores TPU_CORES]
                             [--log_gpu_memory LOG_GPU_MEMORY]
                             [--progress_bar_refresh_rat

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [67]:
import easydict

args = easydict.EasyDict({ "model_params": '/content/drive/MyDrive/cd_ai/s4p/model_best.ckpt' })

In [57]:
# 기존에 저장된 모델 체크포인트 불러오기
model = KoGPT2Chat.load_from_checkpoint(args.model_params)

In [49]:
# pt 파일로 모델 가중치 저장
torch.save(model.state_dict(), 'model_best.pt')
# 모델 불러오기
model.load_state_dict(torch.load('model_best.pt'))

# ckpt 파일 크기 : 1.41GB
# pt 파일 크기 : 489MB

<All keys matched successfully>

이후 다른 곳에서 모델을 불러오려면 아래와 같이 모델 객체 초기화 후 load_state_dict() 메소드를 사용하면 된다.

In [70]:
model = KoGPT2Chat(args)
model.load_state_dict(torch.load('model_best.pt'))

<All keys matched successfully>

In [71]:
model.chat()

user > 잘 됩니까?
Simsimi > 네? 네... 네...
user > quit
